### Libraries import

In [17]:
import pandas as pd # Dataframes management
from zipfile import ZipFile  # Files compressed management
import os # Files management along OS
import re


In [18]:
from selenium import webdriver # Webscrapping bot
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
import time

In [19]:
import nltk
from nltk.tokenize import word_tokenize, sent_tokenize
from nltk.corpus import stopwords

### Data import

In [22]:
# Specifying the name of the zip file
fileZIP = "/items_ordered_2years_V2.zip"
fileCSV = "/items_ordered_2years_V2.csv"

path = "../Inputs/Modificados - Atmira_Pharma_Visualization"
  
# Open the zip file in read mode
with ZipFile(f"{path}{fileZIP}", 'r') as zip: 
    # List all the contents of the zip file
    zip.printdir() 
  
    # Extract all files
    print('extraction...') 
    zip.extractall(path) 
    print('Done!')

#Import CSV to pandas
itemsOrdered = pd.read_csv(f"{path}{fileCSV}")
print("CSV imported to Pandas successfully")

# Remove uncompressed CSV file
os.remove(f"{path}{fileCSV}")
print("Original CSV removed to preserve repo health")

File Name                                             Modified             Size
items_ordered_2years_V2.csv                    2022-02-23 01:28:04    150372498
extraction...
Done!
CSV imported to Pandas successfully
Original CSV removed to preserve repo health


In [23]:
itemsOrdered.head(3)

,num_order,item_id,created_at,product_id,qty_ordered,base_cost,price,discount_percent,customer_id,city,zipcode
0,562ba4ba9aaf2a1e926842deca19271d,2bca87c2ab10b07feef290a9be47f316,2017-01-01 00:20:33,6409.0,1.0,13.4204,17.17,8.0,50d7f99947b472cc889d58845b9d23e2,Valencia,46019
1,562ba4ba9aaf2a1e926842deca19271d,53e12ac4b579f0c4b1c4f77d50bf8792,2017-01-01 00:20:33,5133.0,1.0,18.5351,22.68,8.0,50d7f99947b472cc889d58845b9d23e2,Valencia,46019
2,562ba4ba9aaf2a1e926842deca19271d,d19d6df5cd6a1e741408b70aa2767bf4,2017-01-01 00:20:33,5125.0,1.0,18.5260,22.68,8.0,50d7f99947b472cc889d58845b9d23e2,Valencia,46019


In [ ]:
itemsOrdered[itemsOrdered["city"]=="les botigues de sitges"]

,num_order,item_id,created_at,product_id,qty_ordered,base_cost,price,discount_percent,customer_id,city,zipcode
203989,10a97b452a0d8de97e406d7faadbb7dc,516473bbbe47cbcbd090bed79f835ac0,2017-09-03 20:24:31,5578.0,1.0,2.4551,4.08,5.0,df5602637d380293696349f08f91fa57,les botigues de sitges,08860


**Inviable**

**Se extrae la columna de código postal**

In [ ]:
zipSerie = itemsOrdered["zipcode"]
print(zipSerie.shape)
zipSerie.head(2)

(321671,)


0    46019
1    46019
Name: zipcode, dtype: object

**Se analizan las condiciones de zipcode**

In [ ]:
zipSerie.unique()

array(['46019', '04009', '08225', ..., '4445-151', '43765', '3515-383'],
      dtype=object)

In [ ]:
zipSerie.isna().sum()

1198

**Se comprueba que zipcode tiene valores nulos, se indaga en esta cuestión**

In [ ]:
zipSerie[zipSerie.isna()]

103273    NaN
103274    NaN
103275    NaN
103276    NaN
103492    NaN
         ... 
319351    NaN
319352    NaN
319476    NaN
319477    NaN
319478    NaN
Name: zipcode, Length: 1198, dtype: object

In [ ]:
zipSerie.shape[0]-len(zipSerie[zipSerie.isna()].index)

320473

In [ ]:
round((len(zipSerie[zipSerie.isna()].index)/zipSerie.shape[0])*100, 2)

0.37

**Hay un 37% de valores nulos en la variable. De momento se eliminan con el fin de probar la API para los zipcodes, pero deberá valorarse a futuro**

In [ ]:
zipSerie.dropna(inplace=True)
zipSerie.shape[0]

320473

In [ ]:
itemsOrdered[itemsOrdered["zipcode"]=="30139"]

,num_order,item_id,created_at,product_id,qty_ordered,base_cost,price,discount_percent,customer_id,city,zipcode
30,5992c1625cc4b93264594f98c0c88db3,7fcbb601cef70f0fced43efe096bd879,2017-01-01 08:31:58,41884.0,1.0,8.6242,14.50,8.0,528041acb3b5ddddc51f42c95a78c21a,Murcia,30139
31,5992c1625cc4b93264594f98c0c88db3,9c7846d8386f7191de9640a4115dcb73,2017-01-01 08:31:58,6159.0,1.0,4.9140,7.40,8.0,528041acb3b5ddddc51f42c95a78c21a,Murcia,30139
32,5992c1625cc4b93264594f98c0c88db3,8f29bf49f01d7e6b2d516f8bd6cfe435,2017-01-01 08:31:58,9048.0,1.0,2.3900,2.60,8.0,528041acb3b5ddddc51f42c95a78c21a,Murcia,30139
33,5992c1625cc4b93264594f98c0c88db3,8a4653d26eac38f05519dc529e5b4639,2017-01-01 08:31:58,24172.0,1.0,9.5160,12.14,8.0,528041acb3b5ddddc51f42c95a78c21a,Murcia,30139
34,5992c1625cc4b93264594f98c0c88db3,8a4653d26eac38f05519dc529e5b4639,2017-01-01 08:31:58,24172.0,1.0,9.5160,12.14,8.0,528041acb3b5ddddc51f42c95a78c21a,Murcia,30139
...,...,...,...,...,...,...,...,...,...,...,...
218241,30531d416799775c5c880b6e62784471,1a7bfe6e3a2f4cf5c1e889fbd068e952,2017-09-19 16:33:47,8181.0,1.0,17.4809,20.80,7.0,3e8818a695464113ae8f8cc2780eabf3,EL RAAL,30139
218242,30531d416799775c5c880b6e62784471,8f5504de4dc183ad328ae722e2e0c861,2017-09-19 16:33:47,11788.0,1.0,2.6300,4.09,7.0,3e8818a695464113ae8f8cc2780eabf3,EL RAAL,30139
218243,30531d416799775c5c880b6e62784471,880141d0d213ba805a6c1fb5d39960c1,2017-09-19 16:33:47,30165.0,1.0,5.1800,5.91,7.0,3e8818a695464113ae8f8cc2780eabf3,EL RAAL,30139
218244,30531d416799775c5c880b6e62784471,f039c4a3a5738dceea2c9e940bdb9726,2017-09-19 16:33:47,6368.0,1.0,21.6370,25.74,7.0,3e8818a695464113ae8f8cc2780eabf3,EL RAAL,30139


### USO DE API PARA CÓDIGO POSTAL

Descartado por inviabilidad económica y de tiempo

### USO DE WEBSCRAPPING PARA CÓDIGO POSTAL

In [ ]:
def AcentosLimpiador(text):
	acentos = {'á': 'a', 'é': 'e', 'í': 'i', 'ó': 'o', 'ú': 'u', 'Á': 'A', 'E': 'E', 'Í': 'I', 'Ó': 'O', 'Ú': 'U'}
	for ele in acentos:
		if ele in text:
			text = text.replace(ele, acentos[ele])
	return text

In [ ]:
def CityCleaner(text):
    stopWordSpanish = set(stopwords.words('spanish'))
    wordTokens = word_tokenize(AcentosLimpiador(text.lower()).rstrip()) 
    filteredSentence = [element for element in wordTokens if not element in stopWordSpanish] 
    return filteredSentence

In [ ]:
#Limpieza de zipcodes con RegEx
def num_guion(string):
    """ Get a string with the numbers and hyphens of another string
    
    Args:
        df: string used to extract the string with numbers abd hyphens

    Returns:
        df: the string with numbers and hyphens
    """
    aux = re.match("([\d-]+)", str(string))
    try:
        return str(aux.group())
    except:
        return string


itemsOrdered["zipcode"] = itemsOrdered["zipcode"].apply(lambda x: num_guion(x))

NameError: name 'itemsOrdered' is not defined

In [ ]:
cityToAnalyze = "CARIDAD, LA (EL FRANCO)"
zipToAnalyze = "33750"

In [ ]:
driver = webdriver.Firefox()
    
driver.get("https://worldpostalcode.com/lookup")
driver.set_page_load_timeout(5)

insertZipcode =driver.find_element(By.ID,"search")
insertZipcode.clear()
insertZipcode.send_keys(zipToAnalyze)
clickButtonZipcode =driver.find_element(By.CLASS_NAME,"submit")
clickButtonZipcode.click()
driver.set_page_load_timeout(5)

getGeoInfo = driver.find_element(By.CLASS_NAME,"search_units")
saveGeoInfo = getGeoInfo.text

driver.close()

In [ ]:
listTestingZipcode = saveGeoInfo.split("\n")
listTestingZipcode

['SpainAsturiasLa Caridad33750',
 'SpainAsturiasViavelez33750',
 'SpainAsturiasLongara33750',
 'FranceNouvelle-Aquitaine, GirondeCroignon33750',
 'FranceNouvelle-Aquitaine, GirondeCamarsac33750',
 'FranceNouvelle-Aquitaine, GirondeBeychac33750',
 'FranceNouvelle-Aquitaine, GirondeNerigean33750',
 'FranceNouvelle-Aquitaine, GirondeSaint-Quentin-De-Baron33750',
 'FranceNouvelle-Aquitaine, GirondeBaron33750',
 'FranceNouvelle-Aquitaine, GirondeSaint-Germain-Du-Puch33750',
 'FranceNouvelle-Aquitaine, GirondeCadarsac33750',
 'MexicoChihuahuaAgricola Industrial33750',
 'TurkeyMersin, ErdemliAyas33750']

In [ ]:
indexMatchRegex = list(map(lambda x: [(m.start(0), m.end(0)) for m in re.finditer(r"[a-z][A-Z0-9]", x)], listTestingZipcode))

In [ ]:
len(indexMatchRegex[1])

3

In [ ]:
resultScrapClean = []
for pas,resultScrap in enumerate(listTestingZipcode):
    for pos,ele in enumerate(indexMatchRegex[pas]):
        if len(indexMatchRegex[pas])==2:
            if pos ==0:
                txt = resultScrap[:ele[0]+1]+","+resultScrap[ele[1]-1:]
            elif pos ==1:
                txt = txt[:ele[0]+2]+","+txt[ele[1]:]
                resultScrapClean.append(txt)
        elif len(indexMatchRegex[pas])==3:
            if pos ==0:
                txt = resultScrap[:ele[0]+1]+","+resultScrap[ele[1]-1:]
            elif pos ==1:
                txt = txt[:ele[0]+2]+","+txt[ele[1]:]
            elif pos ==2:
                txt = txt[:ele[0]+3]+","+txt[ele[1]+1:]
                resultScrapClean.append(txt)



In [ ]:
resultScrapClean

['Spain,Asturias,La Caridad,33750',
 'Spain,Asturias,Viavelez,33750',
 'Spain,Asturias,Longara,33750',
 'France,Nouvelle-Aquitaine, Gironde,Croignon,33750',
 'France,Nouvelle-Aquitaine, Gironde,Camarsac,33750',
 'France,Nouvelle-Aquitaine, Gironde,Beychac,33750',
 'France,Nouvelle-Aquitaine, Gironde,Nerigean,33750',
 'France,Nouvelle-Aquitaine, Gironde,Saint-Quentin-De-Baron,33750',
 'France,Nouvelle-Aquitaine, Gironde,Baron,33750',
 'France,Nouvelle-Aquitaine, Gironde,Saint-Germain-Du-Puch,33750',
 'France,Nouvelle-Aquitaine, Gironde,Cadarsac,33750',
 'Mexico,Chihuahua,Agricola Industrial,33750',
 'Turkey,Mersin, Erdemli,Ayas,33750']

In [ ]:
resultScrapListed = []
for element in resultScrapClean:
    resultScrapListed.append(element.split(","))

In [ ]:
resultScrapListed

[['Spain', 'Asturias', 'La Caridad', '33750'],
 ['Spain', 'Asturias', 'Viavelez', '33750'],
 ['Spain', 'Asturias', 'Longara', '33750'],
 ['France', 'Nouvelle-Aquitaine', ' Gironde', 'Croignon', '33750'],
 ['France', 'Nouvelle-Aquitaine', ' Gironde', 'Camarsac', '33750'],
 ['France', 'Nouvelle-Aquitaine', ' Gironde', 'Beychac', '33750'],
 ['France', 'Nouvelle-Aquitaine', ' Gironde', 'Nerigean', '33750'],
 ['France',
  'Nouvelle-Aquitaine',
  ' Gironde',
  'Saint-Quentin-De-Baron',
  '33750'],
 ['France', 'Nouvelle-Aquitaine', ' Gironde', 'Baron', '33750'],
 ['France',
  'Nouvelle-Aquitaine',
  ' Gironde',
  'Saint-Germain-Du-Puch',
  '33750'],
 ['France', 'Nouvelle-Aquitaine', ' Gironde', 'Cadarsac', '33750'],
 ['Mexico', 'Chihuahua', 'Agricola Industrial', '33750'],
 ['Turkey', 'Mersin', ' Erdemli', 'Ayas', '33750']]

In [ ]:
resultScrapRearr = [(element[0], element[1], element[-2], element[-1]) for element in resultScrapListed]

In [ ]:
resultScrapRearr

[('Spain', 'Asturias', 'La Caridad', '33750'),
 ('Spain', 'Asturias', 'Viavelez', '33750'),
 ('Spain', 'Asturias', 'Longara', '33750'),
 ('France', 'Nouvelle-Aquitaine', 'Croignon', '33750'),
 ('France', 'Nouvelle-Aquitaine', 'Camarsac', '33750'),
 ('France', 'Nouvelle-Aquitaine', 'Beychac', '33750'),
 ('France', 'Nouvelle-Aquitaine', 'Nerigean', '33750'),
 ('France', 'Nouvelle-Aquitaine', 'Saint-Quentin-De-Baron', '33750'),
 ('France', 'Nouvelle-Aquitaine', 'Baron', '33750'),
 ('France', 'Nouvelle-Aquitaine', 'Saint-Germain-Du-Puch', '33750'),
 ('France', 'Nouvelle-Aquitaine', 'Cadarsac', '33750'),
 ('Mexico', 'Chihuahua', 'Agricola Industrial', '33750'),
 ('Turkey', 'Mersin', 'Ayas', '33750')]

In [ ]:
resultZip = []
for element in resultScrapRearr:
    for element2  in CityCleaner(element[2]):
        if element2 in CityCleaner(cityToAnalyze):
            resultZip.append(element)
            break
try:
    resultZip = resultZip[0]
except IndexError:
    resultZip = []
    for element in resultScrapRearr:
        for element2 in CityCleaner(element[2]):
            for element3 in CityCleaner(cityToAnalyze):
                if element2.__contains__(element3):
                    resultZip.append(element)
                    break
    resultZip = resultZip[0]

NameError: name 'resultScrapRearr' is not defined